In [7]:
%%javascript

//----------------------------------------------------------------------------------------------------------------------------//

// Access current name of notebook, default = "Untitled"
var kernel         = IPython.notebook.kernel;
var name           = window.document.getElementById("notebook_name").innerHTML;

// Assign notebook name variable for python cell to use
var assign_nb_name = "notebook_name = " + "'" + name + "'";
kernel.execute(assign_nb_name);

//----------------------------------------------------------------------------------------------------------------------------//

// Access user alias of notebook
var user_alias = "";

// 'test' product name to Scout tenant as taken from ProductNameToScoutTenantGuid in [https://sdcdn.azurewebsites.net/Banner.js]
var test_product_name = "7c44b023-8f44-4179-98ce-0ddf5cbe9482";

// Scout API call to access user alias, if not found prompt user to manually enter user alias
$.ajax({
    url: "https://scout/api/logpageview/winauth?tenantGuid=" + test_product_name,
    xhrFields : {
        withCredentials : true
    },
    async : false,
    type: 'GET',
    error: function (response) {
        user_alias = prompt("Unable to retrieve your user alias.\nPlease enter your Microsoft alias below:", "");
    },
    success: function (response) {
        if (response["UserAlias"]) {
            user_alias = response["UserAlias"];
            alert(user_alias);
        }
    }
});

// Assign alias variable for python cell to use
var assign_alias = "ALIAS = " + "'" + user_alias + "'";
kernel.execute(assign_alias);

<IPython.core.display.Javascript object>

In [8]:
from __future__ import print_function
import getpass, json, warnings
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import *
from ipywidgets import Button, HBox, VBox, Layout

# Prompt user for password to alias MS corpnet
PASSWORD = getpass.getpass(prompt="Enter host password for user '" + ALIAS + "':")
warnings.filterwarnings("ignore")

#----------------------------------------------------------------------------------------------------------------------------#

# Determine current python version
PYTHON_VERSION_RAW = !python -V
PYTHON_VERSION     = str(PYTHON_VERSION_RAW[0][7:10])
if PYTHON_VERSION[2] != '5' and PYTHON_VERSION[2] != '6':
    if PYTHON_VERSION[0] == '3':
        PYTHON_VERSION = '3'
    else:
        PYTHON_VERSION = '2'
            
PHILLY_FS = "/mnt/philly-access/philly-fs/python" + PYTHON_VERSION + "/philly-fs.pyc"
HDFS_DIR  = "hdfs://gcr/pnrsy/" + ALIAS

SUDO_USERNAME_RAW = !echo $USER
SUDO_USERNAME     = str(SUDO_USERNAME_RAW)
SUDO_PASSWORD     = getpass.getpass(prompt="Enter host password for user " + SUDO_USERNAME + ":")
!echo $SUDO_PASSWORD | sudo -S mkdir -p /mnt/philly-access 2>/dev/null
!echo $SUDO_PASSWORD | sudo -S mount -t cifs -o username=$ALIAS,domain=REDMOND,password=$PASSWORD,sec=ntlm,vers=2.0 //scratch2/scratch/mradmila /mnt/philly-access 2>/dev/null
!export PHILLY_VC=pnrsy && python $PHILLY_FS -mkdir $HDFS_DIR 2>/dev/null

#----------------------------------------------------------------------------------------------------------------------------#

# MAIN TOOLBAR (includes Docker images, data input directory, advanced button, submit button)

# Formatting layout for main toolbar elements
main_layout = Layout(height='50px')

# Docker Image: to select desired Docker image on Philly
docker_container = Dropdown(
    options = {
        'CNTK'                    : 'cntk', 
        'TensorFlow, Singlebox'   : 'tf-single', 
        'TensorFlow, Distributed' : 'tf-dist',
        'COG1 (Torch)'            : 'cog1', 
        'COG2'                    : 'cog2'
    },
    value = 'tf-single',
    description = 'Docker Image',
    layout=main_layout
)

# Data Input Directory: to enter destination on HDFS server where data is currently stored
input_directory  = Textarea(
    placeholder = '/hdfs/philly/Amit/LargeSpeechTraining', 
    value       = '/hdfs/philly/Amit/LargeSpeechTraining',
    description = 'Data Input Directory',
    layout=main_layout
)

# Advanced Button: to toggle output 
advanced_button  = ToggleButton(value=False, description='Advanced', button_style='info', layout=main_layout)

# Submit Button: to submit job to Philly according to presently registered user-specified selections
submit_button    = Button(description='Submit job to Philly', button_style='danger', layout=main_layout)

main_controls = HBox([docker_container, 
                      input_directory, 
                      advanced_button,
                      submit_button
                     ])

#----------------------------------------------------------------------------------------------------------------------------#

# ADVANCED TOOLBAR (includes job submission name, minimum GPU number, physical cluser, virtual cluster, and various optional opt-ins)

# Job Name: name of job name as shown on Philly Portal
job_name = Text(placeholder=notebook_name, value=notebook_name, description='Job Name')

# Min GPU Number: smallest number of the GPUs you want the job to run with 
gpu_num  = IntSlider(description=' # GPUs', value=1, min=1, max=34)

# Physical Cluster: physical cluster ID that you are submitting the job to
cluster = Dropdown(
    options = {'Cambridge Cluster (cam)'   : 'cam',
               'MSR Grand Central (gcr)'   : 'gcr', 
               'Philly Test Cluster (ccp)' : 'ccp',
               'Redmond Ridge 1 (rr1)'     : 'rr1'
              },
    value = 'gcr',
    description = 'Cluster'
)

# Virtual Clusters: virtual cluster ID that you are submitting the job to
# Virtual clusters on GCR physical cluster
gcr_options = {
    'Bing Ads Group (ipgadsvc)'             : 'ipgadsvc',
    'Bing Speech Recognition Group (ipgsp)' : 'ipgsp',
    'Bing Relevance Group (ipgsrch)'        : 'ipgsrch',
    'Cognitive Services Team (cogsrvc)'     : 'cogsrvc', 
    'Intern Group (intvc)'                  : 'intvc',
    'MSR Asia Imaging Group (msrai)'        : 'msrai',
    'MSR Labs (msrlabs)'                    : 'msr',
    'Philly Express Queue (gcrexpress)'     : 'gcrexpress',
    'Philly Nursery (pnrsry)'               : 'pnrsy',
    'The Research VC (resrchvc)'            : 'resrchvc',
    'Speech Dialog Research Group (sdrgvc)' : 'sdrgvc',
    'WDG Analog Group (anlgvc)'             : 'anlgvc'     
}

# Virtual clusters on RR1 physical cluster
rr1_options = {
    'Ambient Computing Team (xrgpuc)'       : 'xrgpuc',
    'Bing Ads Group (ipgadsvc)'             : 'ipgadsvc',
    'Bing Multimedia team (ipgimg)'         : 'ipgimg',
    'Bing Speech Recognition Group (ipgsp)' : 'ipgsp', 
    'Bing Relevance Team (ipgsrch)'         : 'ingsrch',
    'Cognitive Services Team (cogsrvc)'     : 'cogsrvc',
    'Intern Group (intvc)'                  : 'intvc',
    'MSR Asia Imaging Group (msrai)'        : 'msrai ',
    'MSR Labs (msrlabs)'                    : 'msrlabs',
    'MSR Machine Translation (msrmt)'       : 'msrmt',
    'MSRA Face Reco Team (facevc)'          : 'facevc',
    'Philly Express Queue (rr1express)'     : 'rr1express',
    'Philly Nursery (pnrsy)'                : 'pnrsy',
    'The Research VC (resrchvc)'            : 'resrchvc', 
    'Speech Dialog Research Group (sdrgvc)' : 'sdrgvc', 
    'Text to Speech Team (txt2spch)'        : 'txt2spch', 
    'WDG Analog Group (anlgvc)'             : 'anlgvc', 
    'Windows Evoke Team (evokevc)'          : 'evokevc' 
}

# Virtual clusters on CAM physical cluster
cam_options = { 
    'MSR Labs (msrlabs)' : 'msrlabs' 
}

# Virtual clusters on CCP physical cluster
ccp_options = {
    'Philly Test Cluster Dev changes (phccpdev)'      : 'phccpdev',
    'Philly Test Cluster Dev changes (phccpexpress)'  : 'phccpexpress',
    'Philly Nursery (pnrsy)'                          : 'pnrsy',
    'Philly Test Cluster Prod deployment (phccp)'     : 'phccp', 
    'Philly Test Cluster Test deployment (phccptest)' : 'phccptest'
}

vc = Dropdown(
    options = gcr_options,
    value = 'pnrsy',
    description = 'VC'
)

# Formatting layout for advanced checkboxes
checkbox_box_layout = Layout(width='50px')
checkbox_label_layout = Layout(width='700px', margin='8px 0 0 -25px')

cross_rack       = Checkbox(value=False, layout=checkbox_box_layout)
reserve_nodes    = Checkbox(value=False, layout=checkbox_box_layout)
slow_memcheck    = Checkbox(value=False, layout=checkbox_box_layout)
one_process      = Checkbox(value=False, layout=checkbox_box_layout)

cross_rack_label       = Label('Allow cross rack placements', layout=checkbox_label_layout)
reserve_nodes_label    = Label('Keep the nodes used by this job reserved after job completion for debug purposes', layout=checkbox_label_layout)
slow_memcheck_label    = Label('Turn on slow memcheck for debug purposes', layout=checkbox_label_layout)
one_process_label      = Label('One process per container', layout=checkbox_label_layout)

# Combined toolbar elements
advanced_controls = VBox([HBox([cluster, job_name]),
                          HBox([vc, gpu_num]),
                          VBox([HBox([ cross_rack, cross_rack_label ]),
                                HBox([ reserve_nodes, reserve_nodes_label ]),
                                HBox([ slow_memcheck, slow_memcheck_label ]),
                                HBox([ one_process, one_process_label ])
                               ])
                         ])

#----------------------------------------------------------------------------------------------------------------------------#

# Display controls
display(main_controls)
advanced_controls.layout.display = "none"  # do not initially display advanced controls, another possibility = accordion
display(advanced_controls)

#----------------------------------------------------------------------------------------------------------------------------#

def create_py_script():
    # Save current notebook as config file
    ipynb_name  = notebook_name + ".ipynb"
    py_name     = notebook_name + ".py"
    script_name = notebook_name
    
    !jupyter nbconvert --to python $ipynb_name 2>/dev/null

    # TELL WHETHER OS IS WINDOWS OR NOT (use either 'cat' or 'type') [!]
    !cat $py_name > tmp_0.py
    !rm $py_name
    # print(script_name)
    
    # Make two files recursively, starting from first and second instances of regex respectively
    
    # Parse python script to not contain toolbar elements
    REGEX = "'/# BEGINNING OF CONFIG FILE:/,$p'"
    REPLACEMENT_REGEX = "'0,/# BEGINNING OF CONFIG FILE:/ s/# BEGINNING OF CONFIG FILE:/# NOTHING/'"

    !sed -e $REPLACEMENT_REGEX tmp_0.py > tmp_1.py && rm tmp_0.py
    !sed -e $REPLACEMENT_REGEX tmp_1.py > tmp_2.py && rm tmp_1.py
    !sed -e $REPLACEMENT_REGEX tmp_2.py > tmp_3.py && rm tmp_2.py

    !cat tmp_3.py | sed -n -e $REGEX > "$py_name" #2>/dev/null
    !rm tmp_3.py
    
    return py_name

#----------------------------------------------------------------------------------------------------------------------------#

def add_py_script_to_scratch():
    SCRIPT_NAME = create_py_script()
    !export PHILLY_VC=pnrsy && python $PHILLY_FS -cp $SCRIPT_NAME $HDFS_DIR 2>/dev/null

#----------------------------------------------------------------------------------------------------------------------------#

# Identify and return correct parameters for 
def create_cmd(CMD):
    add_py_script_to_scratch()
        
    tool_type = 'cust'  # defaults for all jobs, assume TensorFlow (singlebox)
    build_id  = '0000'
    custom_docker_name = 'custom-tf-r1-2-ubuntu-16-04-cuda8'
    
    if   (docker_container.value == 'cntk'):                                # CNTK job
        tool_type = 'cntk-py34'
        build_id  = '36069'
    elif (docker_container.value == 'cog1'):                                # COG1 job
        tool_type = 'cog1'
    elif (docker_container.value == 'cog2'):                                # COG2 job
        tool_type = 'cog2'
    elif (docker_container.value == 'tf-dist'):                             # Tensorflow (distributed) job
        tool_type = 'cust'
        custom_docker_name = 'custom-tensorflow-community-gpu-1-2-1-cp35'
    else:                                                                   # Tensorflow (singlebox) job
        tool_type = 'cust'
        custom_docker_name = 'custom-tf-r1-2-ubuntu-16-04-cuda8'
            
    CMD += "\""         # wrap command in quotation marks for curl command submission
    CMD += "https://philly/api/submit?"
    CMD += "buildId=" + build_id + "&"                                                  # Build ID - ...give description
    
    if (docker_container.value != "cntk"):
        CMD += "customDockerName=" + custom_docker_name + "&"                           # Docker image
    
    CMD += "toolType=" + tool_type + "&"                                                # Tool type
    CMD += "clusterId=" + cluster.value + "&"                                           # Cluster
    CMD += "vcId=" + vc.value +"&"                                                      # VC
    CMD += "configFile=" + ALIAS + "%2F" + job_name.value.replace("/", "%2F") + ".py&"  # Submit the cells of this file [!]
    CMD += "minGPUs=" + str(gpu_num.value) + "&"                                        # GPUs
    CMD += "name=" + job_name.value + "&"                                               # Name of test, default = notebook title
    CMD += "iscrossrack=" + str(cross_rack.value).lower() + "&"                         # Cross rack?
    CMD += "isdebug=" + str(reserve_nodes.value).lower() + "&"                          # Reserved nodes after completion?
    CMD += "ismemcheck=" + str(slow_memcheck.value).lower() + "&"                       # Is memcheck?
    CMD += "oneprocesspercontainer=" + str(one_process.value).lower() + "&"             # One process per container?
    CMD += "inputDir=" + input_directory.value.replace("/", "%2F") + "&"                # Input directory
    CMD += "userName=" + ALIAS                                                          # Username
    CMD += "\""
    
    return(CMD)

#----------------------------------------------------------------------------------------------------------------------------#

# Cluster dropdown functionality: depending upon physical cluster selection, 
def on_cluster_selection_click(b):
    vc.value   = 'pnrsy'
    if   (cluster.value == 'gcr') : vc.options = gcr_options
    elif (cluster.value == 'ccp') : vc.options = ccp_options
    elif (cluster.value == 'rr1') : vc.options = rr1_options
    else: 
        vc.options = cam_options
        vc.value   = 'msrlabs'

#----------------------------------------------------------------------------------------------------------------------------#
        
# Advanced button functionality: toggle advanced elements
def on_advanced_button_click(b):
    if advanced_button.value:
        advanced_controls.layout.display = 'flex'
    else:
        advanced_controls.layout.display = 'none'
    
#----------------------------------------------------------------------------------------------------------------------------#
    
# Submit button functionality: submit curl command to Philly via Philly REST API
def on_submit_button_click(b):
    CMD = ''
    CMD = create_cmd(CMD)   # create curl command from user input and/or defaults

    # Call Philly REST API with given parameters
    !curl -k --ntlm --user $ALIAS:$PASSWORD $CMD > response.json 2>/dev/null
    response_raw = !cat response.json
    !rm response.json
    
    # Check that password for correctly authenticated [!]
    if 'Access is denied due to invalid credentials' in str(response_raw):
        print('\nAccess is denied due to invalid credentials.')
        print('Please re-run current cell with valid password for user \'' + ALIAS + "'.")
    else:
        # Parse received response
        # --> Successful job example: ['{"phillyversion": 116, "jobId": "application_1500519950009_8293"}']
        # --> Failed job example: ['{"diag": "", "cmd": "", "phillyversion": 116, "error": "You must use specfic jenkins build number 31323 or greater for now"}']
        response = json.loads(response_raw[0])
    
        # Successful job, report back job submission ID to trace job on Philly portal
        if 'jobId' in response: 
            print("\nJob submitted to Philly")
            print("Your job submission ID is: " + response['jobId'][12:])           # Sample ==> 1500519950009_8293
            
        # Falsed job, report back errors
        elif 'error' in response:
            print("\nJob not submitted to Philly.")
            
            # Check for corpnet access
            if "does not exist on HDFS" in response['error']:
                print("Ensure you have access to MS corpnet.")
                
            print("Error message: " + response['error'] + "; " + response['diag'])  # Sample ==> failed to submit job; could not find file, stack trace(): etc.

#----------------------------------------------------------------------------------------------------------------------------#
 
# Monitor user's toolbar activity async with running of the cell
cluster.observe(on_cluster_selection_click)
advanced_button.observe(on_advanced_button_click, 'value')
submit_button.on_click(on_submit_button_click)

#----------------------------------------------------------------------------------------------------------------------------#

# TEST JOB: "https://philly/api/submit?buildId=0000&customDockerName=custom-docker&toolType=cust&clusterId=gcr&vcId=pnrsy&configFile=t-kabres%2FloopTest.lua&minGPUs=1&name=test40&isdebug=false&iscrossrack=false&inputDir=%2Fhdfs%2Fphilly%2FAmit%2FLargeSpeechTraining&userName=t-kabres"

# TODO: N/A

# BEGINNING OF CONFIG FILE:

Enter host password for user 't-kabres':········
Enter host password for user ['karabressler']:········

Job submitted to Philly
Your job submission ID is: 1500519950009_8293


In [8]:
import tensorflow as tf

if _name_ == "_main_":
    print("Hello World!")
    print(tf._version_)